In [ ]:
# Diagnostic: Check all available GPUs (run this BEFORE setting CUDA_VISIBLE_DEVICES)
# This helps identify which GPU is which
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=index,name,memory.total,compute_cap', '--format=csv,noheader'], 
                       capture_output=True, text=True)
if result.returncode == 0:
    print("Available GPUs:")
    print(result.stdout)
else:
    print("Could not query GPUs with nvidia-smi")


In [ ]:
import os

# MUST be done before ANYTHING imports torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#os.environ["TORCH_USE_CUDA_DSA"] = "1"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

print("CUDA_VISIBLE_DEVICES =", os.environ["CUDA_VISIBLE_DEVICES"])

In [ ]:
import torch
print("Torch sees GPUs:", torch.cuda.device_count())
print("Using device:", torch.cuda.get_device_name(0))

In [ ]:

import torch
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
from datasets import load_dataset
dataset = load_dataset("tedqc/mineral-dataset")

In [ ]:
dataset = dataset['train'].train_test_split(test_size=0.2)

In [ ]:
dataset["train"][0]

In [ ]:
# Get labels - handle both ClassLabel and string features
from datasets import ClassLabel

label_feature = dataset["train"].features["label"]
if isinstance(label_feature, ClassLabel):
    # If it's ClassLabel, get names from feature
    labels = label_feature.names
else:
    # If it's string, get unique labels from the dataset
    labels = sorted(set(dataset["train"]["label"]))

# Create mappings
# label2id: maps label name (string) -> integer ID (for training)
# id2label: maps integer ID -> label name (string) (transformers expects string keys)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i  # Use integer for label2id (needed for training)
    id2label[str(i)] = label  # Use string key for id2label (transformers convention)
    # Also add integer key for consistency checks
    id2label[i] = label

# Verify consistency
print(f"Total labels: {len(labels)}")
print(f"Sample verification (first 5 labels):")
for i in range(min(5, len(labels))):
    label = labels[i]
    assert label2id[label] == i, f"Mismatch: label2id['{label}'] = {label2id[label]}, expected {i}"
    assert id2label[str(i)] == label, f"Mismatch: id2label['{i}'] = {id2label[str(i)]}, expected '{label}'"
    assert id2label[i] == label, f"Mismatch: id2label[{i}] = {id2label[i]}, expected '{label}'"
    print(f"  ✓ {i}: '{label}' -> label2id={label2id[label]}, id2label[{i}]='{id2label[i]}', id2label['{i}']='{id2label[str(i)]}'")
print("✅ All mappings are consistent!")

In [ ]:
print(f"Number of labels: {len(labels)}")
print(f"Number of label2id entries: {len(label2id)}")
print(f"Number of id2label entries: {len(id2label)}")
print(f"\nFirst 10 labels:")
for i in range(min(10, len(labels))):
    label = labels[i]
    print(f"  Label: {label}, label2id: {label2id.get(label, 'MISSING')}, id2label: {id2label.get(i, 'MISSING')}")
    # Verify they match
    if label2id.get(label) != i or id2label.get(i) != label:
        print(f"    ⚠️  MISMATCH!")

# Verify consistency
print(f"\nVerifying consistency...")
all_match = True
for i, label in enumerate(labels):
    if label2id.get(label) != i:
        print(f"Mismatch: label '{label}' -> label2id={label2id.get(label)}, expected {i}")
        all_match = False
    if id2label.get(i) != label:
        print(f"Mismatch: id {i} -> id2label={id2label.get(i)}, expected '{label}'")
        all_match = False

if all_match:
    print("✅ All mappings are consistent!")
else:
    print("❌ Found inconsistencies in mappings")

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "facebook/convnext-base-224"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    # Convert string labels to integer IDs
    if "label" in examples:
        examples["label"] = [label2id[label] if isinstance(label, str) else int(label) for label in examples["label"]]
    del examples["image"]
    return examples

In [ ]:
dataset = dataset.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

# Move model to the correct device
model = model.to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir="mineral_model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    fp16=True,  # Enable half precision (fp16) training
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()